# P-values in ML 02

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sc
import sklearn as sk
import sys
import os

from scipy.stats import chi2
from sklearn.ensemble import RandomForestClassifier
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.multitest import fdrcorrection

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Create dataset

In [2]:
def logit(h):
    ''' Logistic from activation h '''
    p = 1.0 / (1.0 + np.exp(-h))
    r = np.random.rand(len(p))
    y = (r < p).astype('float')
    return y


def rand_date():
    max_time = int(time.time())
    t = random.randint(0, max_time)
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))


def rand_unif(num, mean, std, na_prob=0):
    xi = np.random.rand(num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rand_norm(num, mean=0.0, std=1.0, na_prob=0):
    xi = np.random.normal(mean, std, num)
    if na_prob > 0:
        xi_na = (np.random.rand(num) <= na_prob)
        xi[xi_na] = np.nan
    return xi


def rbf(x, mu=0, sigma=1):
    """ Radial basis function """
    z = (x - mu) / sigma
    return np.exp(-(z*z))


def split_df(df, var_inputs, var_output, train_index, val_index):
    df_train, df_val = df.iloc[train_index], df.iloc[val_index]
    return df_train[var_inputs], df_train[var_output], df_val[var_inputs], df_val[var_output]

In [3]:
def create_dataset_01(num=1000, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    x4 = rand_norm(num)
    x5 = rand_norm(num)
    n = rand_norm(num)
    y = logit(3.0 * x1 - 2.0 * x2 + 1.0 * x3 + 0.5 * n)
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'y': y})
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

In [4]:
def create_dataset_02(num=1000, save=False):
    """ Create dataset y = f(x1, x2, x3) + noise (r* are not used) """
    x1 = rand_norm(num)
    x2 = rand_norm(num)
    x3 = rand_norm(num)
    x4 = rand_norm(num)
    x5 = rand_norm(num)
    n = rand_norm(num)
    y = logit(0.5 * rbf(x1, 1, 1) - 0.3 * rbf(x2, -1, 1) + 0.3 * x3 + 0.5 * n)
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'y': y})
    if save:
        file = 'zzz.csv'
        print(f"Saving dataset to file '{file}'")
        df.to_csv(file, index=False)
    return df

## Feature importance: Logistic regression

In [5]:
def wilks_model_fit(x, y):
    logit_model = Logit(y, x)
    res = logit_model.fit(disp=0)
    return logit_model, res

def wilks_p_value(df, var_output, vars_null):
    model_null, model_null_results = wilks_model_fit(df[vars_null], df[var_output])
    pvalues = dict()
    for c in df.columns:
        if c != var_output and c not in vars_null:
            xnames = list(vars_null)
            xnames.append(c)
            model_alt, model_alt_res = wilks_model_fit(df[xnames], df[var_output])
            if model_alt is None:
                self._error(f"Could not fit alt model for column/s {c}")
                pval = 1.0
            else:
                d = 2.0 * (model_alt_res.llf - model_null_results.llf)
                pval = chi2.sf(d, 1)
            pvalues[c] = pval
    return pd.Series(pvalues)

## Feature importance: ML

### Cross-validations & models

In [6]:
from sklearn.model_selection import KFold


def new_model_fit(x_train, y_train):
    model = RandomForestClassifier(100)
    model.fit(x_train, y_train)
    return model

_id = 0
def next_id():
    global _id
    _id += 1
    return _id


def is_var_rand(var):
    return var is not None and var.startswith('__rand_')


class FeatureImportanceShuffle:
    """ Feature importance by shuffling input values """
    def __init__(self, model, x_train, y_train, x_val, y_val, num_iter=10):
        self.id = next_id()
        self.model = model
        self.x_train, self.y_train, self.x_val, self.y_val = x_train, y_train, x_val, y_val
        self.num_iter = num_iter
        self.score_null = None  # Null score (one score)
        self.scores_alt = dict()  # Alt scores by variable name (multiple scores for each variable)

    def add_score(self, var, score):
        """ Add score for variable 'var' """
        if var not in self.scores_alt:
            self.scores_alt[var] = list()
        self.scores_alt[var].append(score)

    def __call__(self):
        """ Feature importance by shuffling variable 'var' and comparing performance results """
        if not self.score_null:
            self.score_null = self.model.score(self.x_val, self.y_val)
            # print(f"\t\tFeature importance (id={self.id}): Null model {self.scores_null}")
        for var in self.x_train.columns:
            # Perform shuffling 'num_iter' times
            scores = list()
            for i in range(self.num_iter):
                score = self.feature_importance_variable(var)
                scores.append(score)
                self.add_score(var, score)
            scores = np.array(scores)
            # print(f"\t{var}:\tcount: {len(scores)}\tmean: {scores.mean()}\tstd: {scores.std()}")

    def feature_importance_variable(self, var):
        """ Feature importance by shuffling variable 'var' and comparing performance results """
        # Shuffle 'var' and calculate validation score
        x_shuf = self.x_val.copy()
        x_shuf[var] = x_shuf[var].sample(frac=1).values
        score = self.model.score(x_shuf, self.y_val)
        # print(f"\t\tFeature importance (id={self.id}): Variable '{var}, score: {score}'")
        return score

    def get_null(self):
        """ Get null score """
        return self.score_null

    def get_null_and_rand(self, exclude_var=None):
        """ Get null and all '__rand_' scores as a list """
        values = [self.score_null]
        for var in self.x_train.columns:
            if is_var_rand(var) and var != exclude_var:
                values.extend(self.scores_alt[var])
        return values

    def get_alt(self, var):
        """ Get scores for variable 'var', or None if not found """
        return self.scores_alt.get(var)


class FeatureImportance:
    """
    Calculate feature importance.
    """
    def __init__(self, df, var_output, vars_input=None, vars_null=None, new_model_function=new_model_fit, num_cv=5, num_shuffle=10, num_rand=5, max_iter=10):
        self.df_ori = df
        self.var_output, self.vars_input, self.vars_null = var_output, vars_input, vars_null
        if self.vars_input is None:
            self.vars_input = [c for c in df.columns if c != var_output]
        if self.vars_null is None:
            self.vars_null = [c for c in self.vars_input if c.startswith('rand_')]
        self.new_model_function = new_model_function
        self.df = None
        self.pvals_df = None
        self.max_iter =max_iter
        self.num_cv = num_cv
        self.num_rand = num_rand
        self.num_shuffle = num_shuffle
        self.cvs = list()  # List of cross validation items
    
    def __call__(self):
        # initialize
        self.df = self.extend_df_ori()
        self.initialize_cv()
        # Calculate importance by shuffling
        for count_iter in range(self.max_iter):
            self.pvals_df = pd.DataFrame()
            for fis in self.cvs:
                print(f"Iteration: {count_iter}, Cross-validation: {fis.id}")
                fis()
            stop = self.p_values()
            if stop:
                return self.pvals_df
        return self.pvals_df
    
    def cv_iter(self, df):
        """ Create a cross-validation iterator """
        if self.num_cv > 1:
            return KFold(n_splits=self.num_cv).split(df)
        else:
            # No cross validation, split 80% / 20%
            idx = int(0.8 * len(df))
            idx_train = range(0, idx)
            idx_val = range(idx, len(df))
            return [(idx_train, idx_val)]
        
    def extend_df_ori(self):
        """ Extend the data frame by adding random permutations and random variables """
        df = self.df_ori.copy()
        # Create one additional 'rand' column for every variable
        for c in self.df_ori.columns:
            c_rand = f"__rand_{c}"
            self.vars_input.append(c_rand)
            df[c_rand] = df[c].sample(frac=1).values
        # Create one additional 'rand' column for every variable
        for i in range(self.num_rand):
            c_rand = f"__rand_{i}"
            self.vars_input.append(c_rand)
            df[c_rand] = np.random.rand(len(df[c]))
        return df
        
    def get_alts(self, var):
        """ Get alt scores for variable 'var' """
        scores = list()
        for fis in self.cvs:
            ss = fis.get_alt(var)
            if ss is not None:
                scores.extend(ss)
        return np.array(scores)

    def get_nulls(self, var):
        """ Get null scores, excluding variable 'var' """
        scores = list()
        for fis in self.cvs:
            ss = fis.get_null_and_rand(exclude_var=var)
            if ss is not None:
                scores.extend(ss)
        return np.array(scores)

    def initialize_cv(self):
        """ Initialize cross validations """
        for train_index, val_index in self.cv_iter(self.df):
            x_train, y_train, x_validate, y_validate = split_df(self.df, self.vars_input, self.var_output, train_index, val_index)
            model = self.new_model_function(x_train, y_train)
            fis = FeatureImportanceShuffle(model, x_train, y_train, x_validate, y_validate, num_iter=self.num_shuffle)
            self.cvs.append(fis)

    def p_value(self, var, null_scores, alt_scores):
        null_scores, alt_scores = np.array(null_scores), np.array(alt_scores)
        pval = sc.stats.mannwhitneyu(null_scores, alt_scores, alternative='greater')[1]
        df_row = pd.DataFrame({'name': var
                               , 'count_alt': len(alt_scores)
                               , 'mean_alt': alt_scores.mean()
                               , 'std_alt': alt_scores.std()
                               , 'count_null': len(null_scores)
                               , 'mean_null': null_scores.mean()
                               , 'std_null': null_scores.std()
                               , 'p_value': pval}, index=[len(self.pvals_df)])
        self.pvals_df = self.pvals_df.append(df_row)
        return pval

    def p_values(self):
        # Get null scores
        null_scores = list()
        for fis in self.cvs:
            null_scores.extend(fis.get_null_and_rand())
        # Calculate p-value for every variable
        pvals, isrand = list(), list()
        for var in self.vars_input:
            alt_scores = self.get_alts(var)
            null_scores = self.get_nulls(var)
            pval = self.p_value(var, null_scores, alt_scores)
            pvals.append(pval)
            isrand.append(is_var_rand(var))
        pvals = np.array(pvals)
        isrand = np.array(isrand)
        rejected, pvals_fdr = fdrcorrection(pvals)
        self.pvals_df['pvalues_fdr'] = pvals_fdr
        self.pvals_df['pvalues_fdr_reject'] = rejected
        return (rejected & isrand).any()


### P-values analysis

In [7]:
# def pvalues_shuffle_analysis():
#     pvals_df = None
#     for num_cv in [1, 3, 5, 10, 20]:
#         for num_samples in [50, 100, 200, 300, 400, 500, 1000, 2000, 10000]:
#             for num_shuffle in [3, 5, 10, 20, 50, 100]:
#                 pvals_df = p_values_shuffle(num_samples=num_samples, n_rands=5, num_cv=num_cv, num_shuffle=num_shuffle, pvals_df=pvals_df)
#                 print(f"num_cv: {num_cv}\tnum_samples:{num_samples}\tnum_shuffle:{num_shuffle}")
#     pvals_df.to_csv('p_values_shuffle.csv')
#     return pvals_df

In [8]:
num_samples = 10000
df = create_dataset_02(num_samples)
df.head()

,x1,x2,x3,x4,x5,y
0,0.382666,-0.348815,0.280558,0.494661,-0.745136,1.0
1,-0.755279,1.390585,-1.011255,-0.818307,0.330761,1.0
2,-0.167847,0.398118,-0.206344,0.901032,-1.967521,0.0
3,0.558598,0.091097,0.182158,-0.821111,-0.200976,1.0
4,1.111655,0.118912,1.154168,1.323366,-0.379509,1.0


In [9]:
fi = FeatureImportance(df, 'y', num_shuffle=20, num_cv=10 ,new_model_function=new_model_fit)
fi()

Iteration: 0, Cross-validation: 1


KeyboardInterrupt: 